In [1]:
# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Import NLP libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from textblob import TextBlob
import spacy
from gensim import corpora
from gensim.models import LdaModel
from sklearn.feature_extraction.text import CountVectorizer

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

# Set visualization style
sns.set(style="whitegrid")


AttributeError: module 'seaborn' has no attribute 'settheme'

In [ ]:
# Load the dataset
df = pd.read_csv('../data/raw_analyst_ratings.csv')

# Display the first few rows of the dataset
df.head()
